In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import datetime
import itertools
import json
import os
import pickle
from pprint import pprint
import sys

# TF logging control
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import matplotlib.pyplot as plt
import numpy as np

project_root = os.path.abspath('..')
sys.path.append(project_root)

from sleeprnn.data import utils
from sleeprnn.detection import metrics
from sleeprnn.detection.feeder_dataset import FeederDataset
from sleeprnn.helpers.reader import load_dataset
from sleeprnn.common import constants
from sleeprnn.common import checks
from sleeprnn.common import pkeys, viz

viz.notebook_full_width()

In [ ]:
task_mode = constants.N2_RECORD
dataset_name = constants.MASS_SS_NAME
which_expert = 1

params = pkeys.default_params.copy()
dataset = load_dataset(dataset_name, params=params)

In [ ]:
dataset.global_std = 16

# Check page mask from parent dataset

In [ ]:
signal, marks = dataset.get_subject_data(
    dataset.all_ids[0],
    pages_subset=constants.N2_RECORD,
    normalization_mode=constants.N2_RECORD,
    return_page_mask=False)

In [ ]:
signal, marks, page_mask = dataset.get_subject_data(
    dataset.all_ids[0],
    augmented_page=True,
    pages_subset=constants.N2_RECORD,
    normalization_mode=constants.N2_RECORD,
    return_page_mask=True)

In [ ]:
page_mask_mean = page_mask.astype(np.float32).mean(axis=0)
page_mask_mean.shape

In [ ]:
plt.plot(page_mask_mean)
plt.ylim([-0.1, 1.1])
plt.show()

In [ ]:
page_mask_mean[0]

In [ ]:
page_mask_mean[1]

In [ ]:
start_l = []
end_l = []
for subject_id in dataset.train_ids:
    _, _, page_mask = dataset.get_subject_data(
        subject_id,
        augmented_page=True,
        pages_subset=constants.N2_RECORD,
        normalization_mode=constants.N2_RECORD,
        return_page_mask=True)
    page_mask_mean = page_mask.astype(np.float32).mean(axis=0)
    start_border = page_mask_mean[0]
    end_border = page_mask_mean[1]
    # print(start_border, end_border)
    start_l.append(start_border)
    end_l.append(end_border)
print("Mean", np.mean(start_l), np.mean(end_l))
print("Std", np.std(start_l), np.std(end_l))

In [ ]:
output = dataset.get_subject_data(
    dataset.all_ids[0],
    augmented_page=True,
    pages_subset=constants.N2_RECORD,
    normalization_mode=constants.N2_RECORD,
    return_page_mask=True)
for o in output:
    print(o.shape, o.dtype)

In [ ]:
outputs = dataset.get_subset_data(
    dataset.all_ids[:4],
    augmented_page=True,
    pages_subset=constants.N2_RECORD,
    normalization_mode=constants.N2_RECORD,
    return_page_mask=True)
for output in outputs:
    for o in output:
        print(o.shape, o.dtype)

# Check feeder

In [ ]:
subset_ids = dataset.all_ids[:4]
data_feed = FeederDataset(dataset, subset_ids, task_mode, which_expert=which_expert)

In [ ]:
outputs = data_feed.get_data_for_training(return_page_mask=True)
for output in outputs:
    for o in output:
        print(o.shape, o.dtype)